In [2]:
from openocr import OpenRecognizer
from openocr.tools.engine.config import Config
import cv2
import numpy as np

onnx_engine = OpenRecognizer(config=Config("./output/repsvtrch_results/config.yml").cfg, backend='torch')
dummy = [np.random.randint(0, 256, (32, 128, 3), dtype=np.uint8) for _ in range(64)]
onnx_engine(img_numpy_list = dummy, batch_num=64)

img_list = []
for i in range(86):
    img_path = f'../dataset_openocr/test/sample_89_{i}.jpg'
    img = cv2.imread(img_path)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
    img_list.append(img)
    
    
import time
t1 = time.time()
results = onnx_engine(img_numpy_list = img_list, batch_num=64)
print(time.time()-t1)
print("".join([i['text'] for i in results]))


[2025/06/11 00:52:26] openrec INFO: finetune from checkpoint ./output/repsvtrch_results/best.pth
1.0800490379333496
TProject OQuantumn Sentinel: Cybernetic Warfare Strate-Qgies for Countering 2Espionage in the Digital AgeAbstractIen the realm of modern warfare, the threat of espionage in the digital age looms larger than ever betrore. Recognizring this critieal vulnerability Project Quantum Sentinel was devised as a groundbreaktng initiative to develop cybernetic warfare strategies that can effectively counter such threats. ledtnoy the sentient supercomputer CYPHER and its sophisticated petwork of forces, the project delvestento the realms of artilieial intelligence, machine learning, and quantum computing to create unpreceadfented defensive capabilities. By harnessing the power of advanced algorithmns and predictive analyttes, Project Cuantum Sentinel aims to proactively identify and neutralize potential threats before theyenaterialize. This abstract provides a glimpse into the innova

In [67]:
from openocr import OpenRecognizer
from openocr.tools.engine.config import Config
onnx_engine = OpenRecognizer(config=Config("./output/repsvtrch_results/config.yml").cfg, backend='torch')
import time
t1 = time.time()
for i in range(79):
    img_path = f'../dataset_openocr/train/sample_96_{i}.jpg'
    results = onnx_engine(img_path)
    # print(eval(results[0]['text'])[0], end=" ")
print(time.time()-t1)

[2025/06/10 17:41:37] openrec INFO: finetune from checkpoint ./output/repsvtrch_results/best.pth
2.058225393295288


In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from OpenOCR import OpenRecognizer
onnx_engine = OpenRecognizer(mode='server', backend='torch', 
                             config_file='./output/repsvtrch_results/config.yml')
# img_path = '../dataset_openocr/test /sample_2750_7.jpg'
# results = onnx_engine(img_path)
# print(results)
import time
t1 = time.time()
for i in range(79):
    img_path = f'../dataset_openocr/test/sample_91_{i}.jpg'
    results = onnx_engine(img_path)
    print(results[0]['score'][0], end=" ")
print(time.time()-t1)
#     # print([r for r in results])

[2025/06/11 12:32:36] openrec INFO: finetune from checkpoint ./output/repsvtrch_results/best.pth
TAnalyzing the Tactical Advancements of CYPHER: A Sentient Military Supercomputer Abstract he classified military paper delves into the groundbreaking analysis of CYPHER, a highly advance asentient supercomputer at the forefront of military technology. This paper explores the intricate web o frorces under CYPHER's command and its extraordinary tactical capabilities. Through an in-depth ceramination, it uncovers the strategic advantages enabled by CYPHER's autonomous decision-makin Iend vast network integration. The study sheds light on the unprecedented potential for enhancin tenilitary operations and achieving decisive outcomes through the utilization of this powerful artificia ntelligence systerm Introduction e dawn o artificial intelligence has ushered in Ae new era in military technology, with CYPHER ceerging as the vanguard o strategic Iedvanceents. Developed as a sentient military Ssp

In [5]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from OpenOCR import OpenRecognizer
onnx_engine = OpenRecognizer(mode='server', backend='torch', 
                             config_file='./output/repsvtrch_results/config.yml')
# img_path = '../dataset_openocr/test /sample_2750_7.jpg'
# results = onnx_engine(img_path)
# print(results)
import time
t1 = time.time()
img_path = f'./sample.png'
results = onnx_engine(img_path)
print(results[0]['score'][0], end=" ")
print(time.time()-t1)
#     # print([r for r in results])

[2025/06/11 12:41:13] openrec INFO: finetune from checkpoint ./output/repsvtrch_results/best.pth
00R, Biag and Clogperh2 Yl in contras tn the peevion. 0.007287740707397461


In [8]:
""" a modified version of CRNN torch repository https://github.com/bgshih/crnn/blob/master/tool/create_dataset.py """

import os
import lmdb
import cv2

import numpy as np


def checkImageIsValid(imageBin):
    if imageBin is None:
        return False
    imageBuf = np.frombuffer(imageBin, dtype=np.uint8)
    img = cv2.imdecode(imageBuf, cv2.IMREAD_GRAYSCALE)
    imgH, imgW = img.shape[0], img.shape[1]
    if imgH * imgW == 0:
        return False
    return True


def writeCache(env, cache):
    with env.begin(write=True) as txn:
        for k, v in cache.items():
            txn.put(k, v)


def createDataset(inputPath, gtFile, outputPath, checkValid=True):
    """
    Create LMDB dataset for training and evaluation.
    ARGS:
        inputPath  : input folder path where starts imagePath
        outputPath : LMDB output path
        gtFile     : list of image path and label
        checkValid : if true, check the validity of every image
    """
    os.makedirs(outputPath, exist_ok=True)
    env = lmdb.open(outputPath, map_size=1099511627776)
    cache = {}
    cnt = 1

    with open(gtFile, 'r', encoding='utf-8') as data:
        datalist = data.readlines()

    nSamples = len(datalist)
    for i in range(nSamples):
        imagePath, label = datalist[i].strip('\n').split('\t')
        imagePath = os.path.join(inputPath, imagePath)

        # # only use alphanumeric data
        # if re.search('[^a-zA-Z0-9]', label):
        #     continue

        if not os.path.exists(imagePath):
            print('%s does not exist' % imagePath)
            continue
        with open(imagePath, 'rb') as f:
            imageBin = f.read()
        if checkValid:
            try:
                if not checkImageIsValid(imageBin):
                    print('%s is not a valid image' % imagePath)
                    continue
            except:
                print('error occured', i)
                with open(outputPath + '/error_image_log.txt', 'a') as log:
                    log.write('%s-th image data occured error\n' % str(i))
                continue

        imageKey = 'image-%09d'.encode() % cnt
        labelKey = 'label-%09d'.encode() % cnt
        cache[imageKey] = imageBin
        cache[labelKey] = label.encode()

        if cnt % 1000 == 0:
            writeCache(env, cache)
            cache = {}
            print('Written %d / %d' % (cnt, nSamples))
        cnt += 1
    nSamples = cnt-1
    cache['num-samples'.encode()] = str(nSamples).encode()
    writeCache(env, cache)
    print('Created dataset with %d samples' % nSamples)


createDataset("../dataset_openocr", "../dataset_openocr/yolo_gt_test.txt", "../dataset_openocr/yolo_test_lmdb")

Written 1000 / 31595
Written 2000 / 31595
Written 3000 / 31595
Written 4000 / 31595
Written 5000 / 31595
Written 6000 / 31595
Written 7000 / 31595
Written 8000 / 31595
Written 9000 / 31595
Written 10000 / 31595
Written 11000 / 31595
Written 12000 / 31595
Written 13000 / 31595
Written 14000 / 31595
Written 15000 / 31595
Written 16000 / 31595
Written 17000 / 31595
Written 18000 / 31595
Written 19000 / 31595
Written 20000 / 31595
Written 21000 / 31595
Written 22000 / 31595
Written 23000 / 31595
Written 24000 / 31595
Written 25000 / 31595
Written 26000 / 31595
Written 27000 / 31595
Written 28000 / 31595
Written 29000 / 31595
Written 30000 / 31595
Written 31000 / 31595
Created dataset with 31595 samples


In [ ]:
from datasets import load_dataset

# Load a small subset using streaming mode
ds = load_dataset("pixparse/pdfa-eng-wds", split="train", streaming=True)

In [4]:
import json
import base64

# Get the first row/sample
sample = next(iter(ds))

# Print top-level keys
print("Top-level keys:", sample.keys())
print()

# Print the key and URL
print("Sample key:", sample["__key__"])
print("Source URL:", sample["__url__"])
print()

# Preview JSON metadata (prettified)
print("=== JSON Metadata ===")
print(json.dumps(sample["json"], indent=2)[:3000])  # limit to 3000 chars

print("\n=== PDF Preview ===")
pdf_bytes = sample["pdf"]
print(f"PDF file size: {len(pdf_bytes)} bytes")

# Optionally, save PDF to disk
with open(f"{sample['__key__']}.pdf", "wb") as f:
    f.write(pdf_bytes)
    print(f"Saved PDF to: {sample['__key__']}.pdf")

Top-level keys: dict_keys(['__key__', '__url__', 'json', 'pdf'])

Sample key: 6368667
Source URL: hf://datasets/pixparse/pdfa-eng-wds@78af41b722c098baef2a87fab70a15dcf8d77ab7/pdfa-eng-train-0000.tar

=== JSON Metadata ===
{
  "pages": [
    {
      "words": {
        "text": [
          "Health",
          "Smart",
          "Virginia",
          "Sample",
          "Lesson",
          "Plan",
          "Grade",
          "8",
          "-",
          "HP-7",
          "Physical",
          "Health",
          "Disease",
          "Prevention/",
          "Health",
          "Promotion",
          "2020",
          "Virginia",
          "SOLs",
          "Grade",
          "8",
          "Sample",
          "Lesson",
          "Plan:",
          "Stroke",
          "Prevention",
          "Description",
          "Please",
          "see",
          "attached",
          "handout",
          "for",
          "a",
          "lesson",
          "submitted",
          "by",
          "a",